# Nhận dạng thực thể có tên sử dụng sklearn-crfsuite

Trong notebook này, chúng ta huấn luyện mô hình CRF cơ bản cho nhận dạng thực thể có tên trên dữ liệu CoNLL2002 và kiểm tra trọng số của mô hình để xem mô hình học được những gì.

Chúng ta cần các thư viện Python NLTK> 3.x và sklearn-crfsuite, sử dụng Python 3.




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#Clone bộ dữ liệu NER Covid_19
!git clone https://github.com/VinAIResearch/PhoNER_COVID19.git
!pip install sklearn-crfsuite
!pip install scikit-learn

fatal: destination path 'PhoNER_COVID19' already exists and is not an empty directory.


In [6]:
!pip install eli5

In [3]:
import nltk
import sklearn_crfsuite
import eli5

/Users/benphan/opt/anaconda3/envs/ai/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


AttributeError: module 'tensorflow.python.training.experimental.mixed_precision' has no attribute '_register_wrapper_optimizer_cls'

In [ ]:
1. Which is relationship between tables Student and Class
    
    The relationship between table student and table class is n - 1 (many students in a class)
    
2. What is primary key of table Result.
    
    The primary key of the Result table is: Student_id
    
3. List name, gender and teacher name of all students
    
    select a.name, s.gender, c.teacher
    

       from Student s

       inner join Class c on s.class_id = c.id

# 1. Dữ liệu huấn luyện (Training data)

Bộ dữ liệu NER Covid19 chứa danh sách các câu tiếng Việt, với các thực thể có tên được chú giải. Bộ dữ liệu này trích xuất các thông tin liên quan đến đaij dịch Covid_19

In [4]:
# Tiến hành đọc dữ liệu
path_file_train = '/PhoNER_COVID19/data/word/train_word.conll'
path_file_test = '/PhoNER_COVID19/data/word/test_word.conll'


def read_data(path_file):
  f = open(path_file,'r',encoding='utf-8')
  #Tiến hành đọc file dữ liệu kết quả trả về là một danh sách các câu, trong đó mỗi câu là một danh sách các cặp token và nhãn tương ứng của nó.
  ## YOUR CODE HERE
 
  ## YOUR CODE HERE


train_sents = read_data(path_file_train)
test_sents = read_data(path_file_test)

train_sents[0]

FileNotFoundError: [Errno 2] No such file or directory: '/PhoNER_COVID19/data/word/train_word.conll'

# 2. Trích xuất đặc trưng

Định nghĩa các đặc trưng. Hãy trích xuất các đặc trưng khác (lower/title/upper flags, đặc trưng của các từ lân cận - features of nearby words) và chuyển đổi chúng sang định dạng sklear-crfsuite - mỗi một câu phải được chuyển đổi thành một danh sách của từ điển. Đây là một đường cơ sở rất đơn giản; bạn chắc chắn có thể làm tốt hơn.

In [ ]:
def word2features(sent, i):
    word = sent[i][0]

    features = {
        'bias': 1.0,
        'word': word,
        'word.lower()': word.lower(), # Lấy chữ viết thường
        'word.isupper()': word.isupper(), # Có viết hoa
        'word.istitle()': word.istitle(), # Chữ cái đầu mỗi từ là viết hoa
        'word.isdigit()': word.isdigit(), # Có phải số
    }

    # Thêm các features của các từ đằng trước (bias, -1:word, -1:word, -1:word.lower(), -1:word.istitle(), -1:word.isupper() )
    ## START YOUR CODE HERE
   
    ## END YOUR CODE HERE


    # Thêm các features của các từ đằng sau (bias, +1:word, +1:word, +1:word.lower(), +1:word.istitle(), +1:word.isupper() )
    ## START YOUR CODE HERE
    
    ## END YOUR CODE HERE

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

Các đặc trưng được trích xuất từ một token duy nhất có dạng như sau:

In [ ]:
X_train[0]

[{'+1:bias': 1.0,
  '+1:word': 'Đồng_thời',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:word.lower()': ',',
  'BOS': True,
  'bias': 1.0,
  'word': 'Đồng_thời',
  'word.isdigit()': False,
  'word.istitle()': False,
  'word.isupper()': False,
  'word.lower()': 'đồng_thời'},
 {'+1:bias': 1.0,
  '+1:word': ',',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:word.lower()': 'bệnh_viện',
  '-1:bias': 1.0,
  '-1:word': ',',
  '-1:word.istitle()': False,
  '-1:word.isupper()': False,
  '-1:word.lower()': 'đồng_thời',
  'bias': 1.0,
  'word': ',',
  'word.isdigit()': False,
  'word.istitle()': False,
  'word.isupper()': False,
  'word.lower()': ','},
 {'+1:bias': 1.0,
  '+1:word': 'bệnh_viện',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:word.lower()': 'tiếp_tục',
  '-1:bias': 1.0,
  '-1:word': 'bệnh_viện',
  '-1:word.istitle()': False,
  '-1:word.isupper()': False,
  '-1:word.lower()': ',',
  'bias': 1.0,
  'word': 'bệnh_viện',
  'w

# 3. Huấn luyện mô hình CRF

Khi chúng ta có các đặc trưng ở định dạng phù hợp chúng ta có thể huấn luyện mô hình CRF chuỗi tuyến tính (Trường điều kiện ngẫu nhiên - CRFs) bằng cách sử dụng sklearn_crfsuite.CRF:

In [ ]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=20,
    all_possible_transitions=False,
)
crf.fit(X_train, y_train);

# 4. Kiểm tra trọng số mô hình

Các mô hình CRF của CRFsuite sử dụng hai loại đặc trưng: đặc trưng trạng thái và đặc trưng chuyển tiếp. Hãy kiểm tra trọng số của chúng bằng eli5.explain_weights:



In [ ]:
eli5.show_weights(crf, top=20)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


From \ To,O,B-AGE,I-AGE,B-DATE,I-DATE,B-GENDER,B-JOB,I-JOB,B-LOCATION,I-LOCATION,B-NAME,I-NAME,B-ORGANIZATION,I-ORGANIZATION,B-PATIENT_ID,I-PATIENT_ID,B-SYMPTOM_AND_DISEASE,I-SYMPTOM_AND_DISEASE,B-TRANSPORTATION,I-TRANSPORTATION
O,2.595,0.302,0.0,1.401,0.0,0.476,0.468,0.0,1.262,0.0,0.21,0.0,1.297,0.0,0.777,0.0,1.792,0.0,0.461,0.0
B-AGE,0.371,0.0,0.006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-AGE,-0.901,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-DATE,0.229,0.0,0.0,0.0,2.478,0.0,0.0,0.0,0.0,0.0,-0.136,0.0,-0.173,0.0,0.0,0.0,-0.208,0.0,0.0,0.0
I-DATE,0.076,0.0,0.0,0.0,2.295,0.0,0.0,0.0,-0.523,0.0,0.0,0.0,-0.104,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-GENDER,0.172,0.147,0.0,0.0,0.0,0.0,0.161,0.0,0.0,0.0,0.003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-JOB,0.287,0.016,0.0,0.0,0.0,0.002,0.0,0.158,-0.005,0.0,0.077,0.0,0.017,0.0,-0.038,0.0,0.0,0.0,0.0,0.0
I-JOB,-0.334,0.012,0.0,0.0,0.0,0.0,0.0,0.129,-0.018,0.0,0.0,0.0,0.002,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-LOCATION,-0.084,0.0,0.0,-0.644,0.0,0.0,0.0,0.0,0.0,4.724,-0.289,0.0,-0.371,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-LOCATION,-0.129,0.0,0.0,-0.552,0.0,0.0,0.0,0.0,-1.437,4.994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Kiểm tra một phần đặc trưng chuyển tiếp

In [ ]:
eli5.show_weights(crf, top=5, show=['transition_features'])

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


From \ To,O,B-AGE,I-AGE,B-DATE,I-DATE,B-GENDER,B-JOB,I-JOB,B-LOCATION,I-LOCATION,B-NAME,I-NAME,B-ORGANIZATION,I-ORGANIZATION,B-PATIENT_ID,I-PATIENT_ID,B-SYMPTOM_AND_DISEASE,I-SYMPTOM_AND_DISEASE,B-TRANSPORTATION,I-TRANSPORTATION
O,2.595,0.302,0.0,1.401,0.0,0.476,0.468,0.0,1.262,0.0,0.21,0.0,1.297,0.0,0.777,0.0,1.792,0.0,0.461,0.0
B-AGE,0.371,0.0,0.006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-AGE,-0.901,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-DATE,0.229,0.0,0.0,0.0,2.478,0.0,0.0,0.0,0.0,0.0,-0.136,0.0,-0.173,0.0,0.0,0.0,-0.208,0.0,0.0,0.0
I-DATE,0.076,0.0,0.0,0.0,2.295,0.0,0.0,0.0,-0.523,0.0,0.0,0.0,-0.104,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-GENDER,0.172,0.147,0.0,0.0,0.0,0.0,0.161,0.0,0.0,0.0,0.003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-JOB,0.287,0.016,0.0,0.0,0.0,0.002,0.0,0.158,-0.005,0.0,0.077,0.0,0.017,0.0,-0.038,0.0,0.0,0.0,0.0,0.0
I-JOB,-0.334,0.012,0.0,0.0,0.0,0.0,0.0,0.129,-0.018,0.0,0.0,0.0,0.002,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-LOCATION,-0.084,0.0,0.0,-0.644,0.0,0.0,0.0,0.0,0.0,4.724,-0.289,0.0,-0.371,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-LOCATION,-0.129,0.0,0.0,-0.552,0.0,0.0,0.0,0.0,-1.437,4.994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Với all_possible_transitions = True CRF đã học được các trọng số âm lớn cho các chuyển đổi không thể như O -> I-ORG.

# 5. Tùy chỉnh
Bảng trên lớn và khó quan sát, eli5 cung cấp một số tùy chọn để chỉ xem xét một phần của các đặc trưng. Chúng ta có thể chỉ kiểm tra một tập hợp con các nhãn:

In [ ]:
eli5.show_weights(crf, top=10, targets=['O', 'B-LOCATION', 'I-LOCATION'])

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


Một tùy chọn khác là chỉ kiểm tra một số đặc trưng - nó giúp kiểm tra xem một hàm đặc trưng có hoạt động như dự kiến hay không. Ví dụ: hãy kiểm tra cách mô hình sử dụng các đặc trưng khuôn dạng từ bằng cách sử dụng đối số feature_re và ẩn bảng chuyển tiếp:

In [ ]:
eli5.show_weights(crf, top=10, feature_re='^word\.is',
                  horizontal_layout=False, show=['targets'])

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


Weight?,Feature
-2.370,word.isupper()
-2.483,word.istitle()
-2.602,word.isdigit()
Weight?,Feature
+2.081,word.isdigit()
-0.206,word.isupper()
-1.169,word.istitle()
Weight?,Feature
-0.032,word.isupper()
-0.222,word.isdigit()


# 6. Trả kết quả về dạng văn bản
Cũng có thể định dạng kết quả dưới dạng văn bản (có thể xuất ra console):

In [ ]:
expl = eli5.explain_weights(crf, top=5)
print(eli5.format_as_text(expl))

Explained as: CRF

Transition features:
                            O    B-AGE    I-AGE    B-DATE    I-DATE    B-GENDER    B-JOB    I-JOB    B-LOCATION    I-LOCATION    B-NAME    I-NAME    B-ORGANIZATION    I-ORGANIZATION    B-PATIENT_ID    I-PATIENT_ID    B-SYMPTOM_AND_DISEASE    I-SYMPTOM_AND_DISEASE    B-TRANSPORTATION    I-TRANSPORTATION
---------------------  ------  -------  -------  --------  --------  ----------  -------  -------  ------------  ------------  --------  --------  ----------------  ----------------  --------------  --------------  -----------------------  -----------------------  ------------------  ------------------
O                       2.595    0.302    0.000     1.401     0.000       0.476    0.468    0.000         1.262         0.000     0.210     0.000             1.297             0.000           0.777           0.000                    1.792                    0.000               0.461               0.000
B-AGE                   0.371    0.000    0.006 

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


#7. Đánh giá kết quả#

In [ ]:
from sklearn_crfsuite import metrics
#Sử dụng thang đo F_Score để kiểm tra mô hình
labels = list(crf.classes_)
labels.remove('O')
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

0.8000596334115092

In [ ]:
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                       precision    recall  f1-score   support

                B-AGE      0.891     0.888     0.890       582
                I-AGE      0.000     0.000     0.000         6
               B-DATE      0.928     0.845     0.885      1654
               I-DATE      0.946     0.985     0.965      1750
             B-GENDER      0.901     0.868     0.884       462
                B-JOB      0.000     0.000     0.000       173
                I-JOB      0.000     0.000     0.000       114
           B-LOCATION      0.844     0.875     0.859      4441
           I-LOCATION      0.819     0.824     0.821      4918
               B-NAME      0.699     0.723     0.711       318
               I-NAME      0.000     0.000     0.000        13
       B-ORGANIZATION      0.884     0.534     0.666       771
       I-ORGANIZATION      0.874     0.534     0.663      2014
         B-PATIENT_ID      0.877     0.943     0.909      2005
         I-PATIENT_ID      0.000     0.000     0.000  

Cài đặt trình Tokenizer

In [ ]:
# Install the vncorenlp python wrapper
!pip3 install vncorenlp

# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter) 
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 5.1 MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645951 sha256=c6a5214494a82bc6aea836effa870a54d8097e85a16008a6311489680b1992ca
  Stored in directory: /root/.cache/pip/wheels/0c/d8/f2/d28d97379b4f6479bf51247c8dfd57fa00932fa7a74b6aab29
Successfully built vncorenlp
--2022-06-23 20:12:11--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.1.1.jar’

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M   166MB/s    in 0.2s    

2022-06

In [ ]:
#Viết hàm có chuỗi đầu vào là một câu đầu ra là các nhãn NER tương ứng với từng Token

from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("/content/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')

def Predictner(text):
  ## START YOUR CODE HERE

  ## END YOUR CODE HERE

text = 'Bác sĩ Nguyễn Trung Nguyên, Giám đốc Trung tâm Chống độc, Bệnh viện Bạch Mai, cho biết bệnh nhân được chuyển đến bệnh viện ngày 7/3, chuẩn đoán ngộ độc thuốc điều trị sốt rét chhloroquine.'

print(Predictner(text))

[('Bác_sĩ', 'O'), ('Nguyễn_Trung_Nguyên', 'O'), (',', 'O'), ('Giám_đốc', 'O'), ('Trung_tâm', 'B-LOCATION'), ('Chống', 'I-LOCATION'), ('độc', 'I-LOCATION'), (',', 'O'), ('Bệnh_viện', 'B-LOCATION'), ('Bạch_Mai', 'I-LOCATION'), (',', 'O'), ('cho', 'O'), ('biết', 'O'), ('bệnh_nhân', 'O'), ('được', 'O'), ('chuyển', 'O'), ('đến', 'O'), ('bệnh_viện', 'O'), ('ngày', 'O'), ('7/3', 'B-DATE'), (',', 'O'), ('chuẩn', 'O'), ('đoán', 'O'), ('ngộ_độc', 'O'), ('thuốc', 'O'), ('điều_trị', 'O'), ('sốt_rét', 'O'), ('chhloroquine', 'O'), ('.', 'O')]
